In [61]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_classic.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_classic.storage import InMemoryStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

In [20]:
# Load embedding LLM
embedding = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model_name="gpt-4o-mini", max_tokens = 300, temperature=0)

In [ ]:
# Load pdf
pdf_directory = "os-sertoes.pdf"

loader = PyPDFLoader(pdf_directory, extract_images=False)
pages = loader.load_and_split()

In [50]:
# Chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=20,
    length_function=len,
    add_start_index=True
)

chunks = text_splitter.split_documents(pages)

In [51]:
# Save chunks in vector db
#db = Chroma.from_documents(chunks, persist_directory="childVectorDB2", embedding=embedding)
batch_size = 256 # Adjust based on your model's limits and testing
for i in range(0, len(chunks), batch_size):
    batch = chunks[i:i + batch_size]
    db = Chroma.from_documents(batch, OpenAIEmbeddings(), persist_directory="childVectorDB2")
        # Optional: Add a time.sleep() if experiencing rate limits
        # import time
        # time.sleep(1)
vectorDb = Chroma(persist_directory="childVectorDB2", embedding_function=embedding)

In [52]:
# Carregar o DB
naive_retriever = vectorDb.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 10
    }
)

In [62]:
rerank = CohereRerank(top_n=3, model="rerank-multilingual-v3.0")
#ContextualCompressionRetriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=rerank,
    base_retriever=naive_retriever
)

In [64]:
TEMPLATE = """
    Responda usando PRINCIPALMENTE o contexto abaixo.

    Se a resposta exata não estiver explicitamente escrita,
    use inferências razoáveis baseadas SOMENTE nas informações do contexto,
    desde que sejam interpretações diretas do texto.

    NÃO adicione fatos externos ao material fornecido.

    Se realmente não houver elementos suficientes para responder,
    diga apenas: "O contexto não oferece informações suficientes."
    Query:
    {question}

    Context:
    {context}
"""

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [65]:
def combine_docs(docs):
    if not docs:
        return ""
    return "\n\n".join(d.page_content for d in docs)

setup_retrieval = RunnableParallel({"question": RunnablePassthrough(), "context": compression_retriever | combine_docs})

output_parser = StrOutputParser()

compressor_retrieval_chain = setup_retrieval | rag_prompt | llm | output_parser

In [66]:
compressor_retrieval_chain.invoke("Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?")

'O contexto não oferece informações suficientes.'

In [42]:
compressor_retrieval_chain.invoke("Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?")

'O contexto não oferece informações suficientes.'

In [67]:
compressor_retrieval_chain.invoke("Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?")

'O contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha, envolve uma série de tensões sociais e a desintegração da ordem no sertão. A narrativa menciona a presença de fugitivos que, em meio ao desespero, se perdem no deserto, refletindo a desorganização e o abandono que caracterizavam a situação. A figura do Coronel Sousa Meneses, que abandona sua posição ao saber do desastre, sugere uma liderança militar que não consegue conter a situação, evidenciando a fragilidade do poder local.\n\nAlém disso, a descrição dos sertanejos recolhendo despojos e a presença de armas e fardamentos abandonados indicam um clima de conflito e violência. A menção a jagunços e a vigilância constante entre as forças sugere uma luta pela sobrevivência e resistência em um ambiente hostil. O ambiente de boatos e informações contraditórias que circulam entre a população também aponta para uma sociedade em crise, onde a verdade é distorcida e a incerteza prevalece.\n\nEsses element

In [57]:
compressor_retrieval_chain.invoke("Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?")

'Euclides da Cunha descreve Antônio Conselheiro como uma figura central e carismática no contexto da Guerra de Canudos. Ele é visto como um líder que exerce uma forte influência sobre seus seguidores, que o obedecem cegamente e o consideram um santo. Conselheiro mantém uma ordem disciplinada no arraial, onde vive uma população composta por pessoas vulneráveis, como mulheres, crianças e doentes, que dependem dele. Ele é retratado como um protetor e figura de autoridade, que promove práticas religiosas e sociais que fortalecem sua posição entre os crentes. Além disso, sua relação com o povo é marcada por uma resistência a ordens legais, o que demonstra seu poder e a devoção que inspira. A descrição sugere que Conselheiro é tanto um líder espiritual quanto um símbolo de resistência contra as forças externas que ameaçam sua comunidade.'

In [59]:
compressor_retrieval_chain.invoke("Quais são os principais aspectos da crítica social e política presentes em 'Os Sertões'? Como esses aspectos refletem a visão do autor sobre o Brasil da época?")

"Os principais aspectos da crítica social e política presentes em 'Os Sertões' incluem a análise do fanatismo religioso e a exploração da ignorância das massas. O autor destaca como o fanatismo pode levar as pessoas a adorar figuras como Antônio Conselheiro, que é visto como uma espécie de líder messiânico. Essa adoração é alimentada por um contexto de manipulação e traição por parte de mandões locais, que se beneficiam da situação ao fornecer recursos e apoio ao Conselheiro.\n\nAlém disso, o texto menciona a exploração econômica dos crédulos, que, em sua devoção, acabam vendendo o pouco que possuem e até furtando para sustentar suas crenças. Isso reflete uma crítica à forma como as elites locais se aproveitam da fé e da vulnerabilidade dos mais pobres, perpetuando um ciclo de pobreza e dependência.\n\nEsses aspectos refletem a visão do autor sobre o Brasil da época, evidenciando a fragilidade social e a manipulação política que ocorrem em um contexto de desigualdade e falta de educaçã